# Finjustering af Open AI-modeller

Denne notebook er baseret på den aktuelle vejledning i [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) dokumentationen fra Open AI.

Finjustering forbedrer ydeevnen af grundmodeller til din applikation ved at genoptræne den med ekstra data og kontekst, der er relevante for det specifikke brugsscenarie. Bemærk, at prompt engineering-teknikker som _few shot learning_ og _retrieval augmented generation_ giver dig mulighed for at forbedre standardprompten med relevante data for at øge kvaliteten. Disse metoder er dog begrænset af det maksimale token-vindue for den valgte grundmodel.

Med finjustering genoptræner vi faktisk selve modellen med de nødvendige data (hvilket gør det muligt at bruge langt flere eksempler, end der kan være i det maksimale token-vindue) – og vi udruller en _tilpasset_ version af modellen, som ikke længere behøver at få eksempler ved inferenstid. Det forbedrer ikke kun effektiviteten af vores promptdesign (vi får mere fleksibilitet til at bruge token-vinduet til andre formål), men kan også potentielt sænke vores omkostninger (ved at reducere antallet af tokens, vi skal sende til modellen ved inferenstid).

Finjustering består af 4 trin:
1. Forbered og upload træningsdataene.
1. Kør træningsjobbet for at få en finjusteret model.
1. Evaluer den finjusterede model og iterér for at opnå kvalitet.
1. Udrul den finjusterede model til inferens, når du er tilfreds.

Bemærk, at ikke alle grundmodeller understøtter finjustering – [tjek OpenAI-dokumentationen](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) for den nyeste information. Du kan også finjustere en model, der tidligere er blevet finjusteret. I denne vejledning bruger vi `gpt-35-turbo` som vores målgrundmodel til finjustering.

---


### Trin 1.1: Forbered dit datasæt

Lad os bygge en chatbot, der hjælper dig med at forstå det periodiske system ved at besvare spørgsmål om et grundstof med et limerick. I _denne_ enkle vejledning laver vi blot et datasæt til at træne modellen med nogle få eksempler på svar, der viser det forventede format for dataene. I en virkelig situation ville du skulle lave et datasæt med mange flere eksempler. Du kan måske også bruge et åbent datasæt (for dit anvendelsesområde), hvis et sådant findes, og omformatere det til brug ved finjustering.

Da vi fokuserer på `gpt-35-turbo` og ønsker et enkelt svar (chat completion), kan vi lave eksempler ved at bruge [dette foreslåede format](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst), som afspejler OpenAIs krav til chat completion. Hvis du forventer samtaler med flere beskeder, skal du bruge [multi-turn eksempelformatet](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst), som inkluderer en `weight`-parameter til at angive, hvilke beskeder der skal bruges (eller ikke bruges) i finjusteringsprocessen.

Vi bruger det enklere single-turn format i denne vejledning. Dataene er i [jsonl-formatet](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) med 1 post pr. linje, hvor hver post er et JSON-objekt. Uddraget nedenfor viser 2 poster som eksempel – se [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) for det fulde eksempelsæt (10 eksempler), vi bruger til vores finjusteringsvejledning. **Bemærk:** Hver post _skal_ være defineret på én linje (ikke delt over flere linjer, som det ofte ses i et formateret JSON-dokument)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

I en virkelig situation skal du bruge et meget større eksempelsæt for at opnå gode resultater – afvejningen vil være mellem kvaliteten af svarene og tid/omkostninger til finjustering. Vi bruger et lille sæt, så vi hurtigt kan gennemføre finjusteringen og illustrere processen. Se [dette OpenAI Cookbook-eksempel](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) for en mere avanceret finjusteringsvejledning.


### Trin 1.2 Upload dit datasæt

Upload dataene ved hjælp af Files API [som beskrevet her](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Bemærk, at for at kunne køre denne kode, skal du først have gjort følgende:
 - Installeret Python-pakken `openai` (sørg for at bruge en version >=0.28.0 for de nyeste funktioner)
 - Sat miljøvariablen `OPENAI_API_KEY` til din OpenAI API-nøgle
Hvis du vil vide mere, kan du se [Opsætningsguiden](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst), som er lavet til kurset.

Kør nu koden for at oprette en fil til upload fra din lokale JSONL-fil.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### Trin 2.1: Opret finjusteringsjobbet med SDK'en



In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### Trin 2.2: Tjek status for jobbet

Her er nogle ting, du kan gøre med `client.fine_tuning.jobs` API'en:
- `client.fine_tuning.jobs.list(limit=<n>)` - Vis de seneste n finetuning-jobs
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Hent detaljer om et specifikt finetuning-job
- `client.fine_tuning.jobs.cancel(<job_id>)` - Annuller et finetuning-job
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Vis op til n hændelser fra jobbet
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Det første trin i processen er _at validere træningsfilen_ for at sikre, at dataene har det rigtige format.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### Trin 2.3: Spor hændelser for at følge fremskridt


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Trin 2.4: Se status i OpenAI-dashboardet


Du kan også se status ved at besøge OpenAI’s hjemmeside og gå til sektionen _Finjustering_ på platformen. Her kan du se status for det aktuelle job og følge historikken for tidligere jobkørsler. På dette skærmbillede kan du se, at den tidligere kørsel mislykkedes, mens den anden kørsel lykkedes. Til info skete dette, fordi den første kørsel brugte en JSON-fil med forkert formaterede poster – da det blev rettet, blev den anden kørsel gennemført uden problemer, og modellen blev gjort tilgængelig til brug.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.da.png)


Du kan også se statusmeddelelser og målinger ved at rulle længere ned i det visuelle dashboard som vist her:

| Beskeder | Målinger |
|:---|:---|
| ![Beskeder](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.da.png) |  ![Målinger](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.da.png)|


### Trin 3.1: Hent ID & Test finjusteret model i kode


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### Trin 3.2: Indlæs & test det finjusterede model i Playground

Du kan nu teste den finjusterede model på to måder. Først kan du besøge Playground og bruge Models-dropdownmenuen til at vælge din nyligt finjusterede model fra listen over muligheder. Den anden mulighed er at bruge "Playground"-valget, som vises i Fine-tuning-panelet (se skærmbilledet ovenfor), hvilket åbner følgende _sammenligningsvisning_, hvor både foundation- og finjusterede modelversioner vises side om side for hurtig evaluering.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.da.png)

Udfyld blot systemkonteksten, som du brugte i dine træningsdata, og indtast dit testspørgsmål. Du vil bemærke, at begge sider opdateres med den samme kontekst og det samme spørgsmål. Kør sammenligningen, og du vil se forskellen i output mellem dem. _Bemærk hvordan den finjusterede model leverer svaret i det format, du har angivet i dine eksempler, mens foundation-modellen blot følger systemprompten_.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.da.png)

Du vil også bemærke, at sammenligningen viser antallet af tokens for hver model samt den tid, det tager at lave inferensen. **Dette specifikke eksempel er meget simpelt og er kun ment som en demonstration af processen, men afspejler ikke et reelt datasæt eller scenarie**. Du kan se, at begge eksempler viser det samme antal tokens (systemkontekst og brugerprompt er identiske), men at den finjusterede model bruger længere tid på inferensen (brugerdefineret model).

I virkelige scenarier vil du ikke bruge et så simpelt eksempel som dette, men i stedet finjustere på baggrund af rigtige data (f.eks. et produktkatalog til kundeservice), hvor kvaliteten af svaret vil være langt mere tydelig. I _den_ sammenhæng vil det kræve mere avanceret prompt engineering at opnå samme svar-kvalitet med foundation-modellen, hvilket vil øge tokenforbruget og potentielt også den tid, det tager at lave inferensen. _Hvis du vil prøve dette, kan du finde eksempler på finjustering i OpenAI Cookbook og komme i gang der._



---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på nøjagtighed, skal du være opmærksom på, at automatiske oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel, menneskelig oversættelse. Vi påtager os intet ansvar for misforståelser eller fejltolkninger, der måtte opstå ved brug af denne oversættelse.
